In [ ]:
function initialize_BP()
    
    if BP_INIT_VERSION == "artificielle" || BP_INIT_VERSION == "heuristique"
        # Ajout d'une colonne artificielle pour initialer l'algorithme de branchement
        artifcolumn = Dict{String,Any}() 
        artifcolumn["cost"] = minimum(WEIGHTS)[2]
        artifcolumn["vertices"] = ones(Int,nv(GRAPH))
        artifcolumn["oneedges"] = []
        push!(column_pool, artifcolumn)
    end
    if BP_INIT_VERSION == "heuristique"
        init_cycles = KEP_heuristic()
        for cycle in init_cycles
            column = Dict{String,Any}() 
            column["cost"] = sum(WEIGHTS[(cycle[i],cycle[i+1])] for i in 1:length(cycle)-1)
            column["vertices"] = zeros(Int,nv(GRAPH))
            column["vertices"][cycle] .= 1
            column["oneedges"] = [(cycle[i],cycle[i+1]) for i in 1:length(cycle)-1]
            push!(column_pool, column)
        end
    end
end

In [ ]:
# Heuristique utilisé au début de B&P
function KEP_heuristic()
    cycles = Array{Array{Int,1},1}() # Contiendra les cycles de cette solution
    
    nodes_to_treat = collect(VERTICES)
    # ranger les noeuds dans un ordre d'importance
    
    for i in nodes_to_treat
        
        queue = Array{Array{Int,1},1}() # cycles en cours de création
        
        dest = [k for k in outneighbors(GRAPH,i) if k in nodes_to_treat] # sans utiliser des sommets déjà compris dans un cycle
        for j in dest
            push!(queue,[j])
        end
        
        while length(queue) > 0
            current = pop!(queue)
            if length(current) < L
                dest = [k for k in outneighbors(GRAPH,current[end]) if k in nodes_to_treat] # sans utiliser des sommets déjà compris dans un cycle
                for j in dest
                    if j == i         
                        cycle = vcat(current,i)         
                        push!(cycles, vcat(i,cycle))
                        for k in cycle
                            deleteat!(nodes_to_treat,findfirst(x -> x == k, nodes_to_treat))
                        end
                        
                        @goto next_vertex
                    end
                    push!(queue, vcat(current,j))
                end
            end  
        end
        
        @label next_vertex  
    end
    
    return cycles
end

In [ ]:
function create_nodes(sol_node,current)
    i,j = sol_node
    
    if BRANCHING_VERSION == "sur_arcs"
        println("Deux noeuds sont créés en branchant sur x_($i,$j)")
        push!(tree, TreeNode(current,[],tree[current].ub,vcat((i,j),tree[current].setzero),tree[current].setone,[],[]))
        push!(tree[current].children, length(tree))
        push!(Queue, length(tree))
        push!(tree, TreeNode(current,[],tree[current].ub,tree[current].setzero,vcat((i,j),tree[current].setone),[],[]))
        push!(tree[current].children, length(tree))
        push!(Queue, length(tree))
        
    elseif BRANCHING_VERSION == "sur_sommets"
        println("Deux noeuds sont créés en branchant sur y_$i == y_$j et y_$i != y_$j")
        push!(tree, TreeNode(current,[],tree[current].ub,[],[],vcat((i,j),tree[current].setequal),tree[current].setdiff))
        push!(tree[current].children, length(tree))
        push!(Queue, length(tree))
        push!(tree, TreeNode(current,[],tree[current].ub,[],[],tree[current].setequal,vcat((i,j),tree[current].setdiff)))
        push!(tree[current].children, length(tree))
        push!(Queue, length(tree))
    end
end

In [ ]:
function Processing_Rule() 
    if PROCESSING_RULE == "profondeur"
        return Queue[end]
        
    elseif PROCESSING_RULE == "largeur"
        return Queue[1]
        
    elseif PROCESSING_RULE == "meilleur_d_abord"
        min = tree[1].ub
        Qi = Queue[1]
        for i in Queue
            if tree[i].ub <= min
                Qi = i
            end
        end
        return Qi
    end
end

In [ ]:
function solve_BP()
    global column_pool = Array{Dict{String,Any},1}()
    # 'column_pool' est l'ensemble de toutes les colonnes ajoutées au problème maître restreint
    # Le premier indice correspond à l'indice de colonne c.
    # Dans la colonne column_pool[c] se trouve un dictionnaire contenant deux éléments :
    #  - "cost"     : cout de la colonne
    #  - "vertices" : liste contenant les valeurs des variables y_i, où i est un couple donneur/receveur
    #  - "oneedges" : liste des arcs du graphe qui sont conservés dans ce cycle (une colonne représente un cycle)
    
    # Initialisation du Branch and Price (colonnes artificielles, utilisation d'une heuristic)
    initialize_BP()
    
    
    # Bornes inférieures et supérieures du problème de départ, mise pour commencer à -infini et +infini
    global LB = -Inf
    global UB = Inf

    global Queue = Vector{Int}() # Contient les indices des noeuds qui doivent être encore être traités
    global tree = Vector{TreeNode}() # Arbre de branchement (sous forme de liste) qui contiendra des objets TreeNode
    
    
    # L'arbre de branch and price est initialisé avec un noeud racine numéro 1
    #   Le noeud racine est son propre parent, il n'a pas encore d'enfants, à pour borne supérieure +infini,
    #   et n'a aucune contrainte de branchement
    push!(tree,TreeNode(0,[],Inf,[],[],[],[]))   
    push!(Queue,1)  # ajout de l'indice 1 qui correspond au noeud racine, et qu'il reste à traité pour l'instant
    
    
    # Tant qu'il reste des à traiter, on branche
    while length(Queue) > 0
        println("Liste des noeuds qu'il reste à traiter : ", Queue)

        current = Processing_Rule() # noeud que l'on traite dans cette boucle

        sol_node = Process_Node(current) # Résoud le noeud courant en calculant sa borne duale (supérieure ici)
                                  # et retourne la liste des variables x_ij pour tous les arcs
                                  # ou bien une liste vide [] si le noeud est irréalisable
      
        # Si le noeud courant est réalisable et que sa borne supérieure est prometteuse alors on branche
        if sol_node != [] && tree[current].ub >= LB
            
            # Si la solution n'est pas entière, alors on branche sur l'arc (i,j) trouvé
            if sol_node != (0,0)
                create_nodes(sol_node,current)  # création des 2 noeuds enfants
                
            # sinon, si la solution est entière, alors on a résolu cette branche de l'arbre
            else
                println(" \e[93m|\e[92m  La solution relâchée optimale était entière")
                println(" \e[93m|\e[92m  Valeur réalisable $(tree[current].ub) trouvée\e[00m")
            end
        
        # sinon, si le noeud est irréalisable ou non-prometteur, alors on l'élague
        else
            println(" \e[93m|\e[31m  Ce noeud est élagué car irréalisable ou car sa borne est non-prometteuse \e[00m")
        end
        
        # Calcul de la borne supérieure globale comme le maximum des bornes supérieures de tous les noeuds actifs
        global UB = tree[current].ub
        for i in Queue
            if tree[i].ub >= UB
                global UB = tree[i].ub
            end
        end
        
        # Affichage des bornes inf et sup globales à cet instant
        println("\e[94mLB=$LB, UB=$UB \e[00m")
        
        # on retire le noeud traité de la liste 'Queue'
        deleteat!(Queue,findfirst(a -> a == current, Queue))
        # on retire tous les noeuds dont la borne supérieure n'est pas prometteuse
        deleteat!(Queue,unique(nodestobedeleted))
        
        # Si le gap d'optimalité est assez faible alors on s'arrête
        if abs(2*(UB-LB)/(UB+LB))<=ϵ
            break
        end
    end
    return length(tree)
end